In [1]:
import pandas
from openpyxl import load_workbook
import numpy as np
import re
import spacy
from cleantext import clean
from unidecode import unidecode
import string

# Funcoes Genericas

##### RENOMEANDO COLUNAS

In [2]:
def renomear_colunas_por_lista(dataframe, novos_nomes):
    # Verificar se o número de novos nomes coincide com o número de colunas no DataFrame
    if len(novos_nomes) != len(dataframe.columns):
        raise ValueError("O número de novos nomes deve ser igual ao número de colunas no DataFrame.")

    # Atribuir os novos nomes às colunas
    dataframe.columns = novos_nomes

    return dataframe

In [3]:
def renomear_colunas_tratando_nome(dataframe):
    df_novo = dataframe.copy()

    for coluna_original in dataframe.columns:
        # tirando acentuacao e colocando em caixa baixa
        valor = unidecode(str(coluna_original)).lower()
    
        # Substituir "/" por "_"
        valor = valor.replace('/', '_')
    
        # Substituir palavras acentuadas por não acentuadas
        valor = unidecode(valor)
    
        # Colocar em minúsculo e substituir espaços por "_"
        valor = valor.lower().replace(' ', '_')
    
        # Substituir "e ou" por "e_ou"
        valor = valor.replace('_eou_', '_e_ou_')
    
        # Substituir "e ou" por "e_ou"
        valor = valor.replace('__', '_')
    
        valor = valor[:-1] if valor[-1] == '_' else valor

        # Remover pontuações, exceto o caractere "_"
        pontuacoes = string.punctuation.replace("_", "")
        valor = valor.translate(str.maketrans("", "", pontuacoes))

    
        # Renomear a coluna
        df_novo = df_novo.rename(columns={coluna_original: valor})

    return df_novo

##### Limpa Texto

In [4]:
 # Converte para minúsculas e Remove acentuacao e pontuacoes para facilitar a comparação
def limpa_texto(texto):
    texto = unidecode(str(texto)).lower()
    #texto = ''.join(char for char in texto if (char not in string.punctuation) & (char != '|'))
    # Criando uma tabela de tradução para substituir pontuações por espaços
    traducao = str.maketrans('', '', string.punctuation.replace('|', ''))
    # Aplicando a tradução e mantendo o caractere '|'
    texto = texto.translate(traducao)
    texto = texto.replace('\n', ' | ').replace('|', ' | ').replace('  ', ' ')
    return texto

In [5]:
# Criando nova coluna no df outiliers com o nome das atividades recebendo o mesmo tratamento que o nome dos campos
def tratar_valor(valor):
    # Remover espaços vazios duplos
    valor = ' '.join(valor.split())

    # Substituir "/" por "_"
    valor = valor.replace('/', '_')

    # Substituir palavras acentuadas por não acentuadas
    valor = unidecode(valor)

    # Colocar em minúsculo e substituir espaços por "_"
    valor = valor.lower().replace(' ', '_')

    # Substituir "e ou" por "e_ou"
    valor = valor.replace('_eou_', '_e_ou_')

    return valor

In [6]:
# Trata os campos que deveriam conter valor em minutos e estao em outros formatos convertendo todos para minutos
# e campos de semanas e meses para inteiro
def calcula_minutos(texto):
    # Converte para minúsculas para facilitar a comparação, Remove acentuacao e Aplicar o processamento spaCy ao texto
    texto = unidecode(str(texto)).lower()
    match = re.match(r'(\d+)([a-z]+)(\d+)?', texto, re.IGNORECASE)
    match_hora_minuto = re.match(r'(\d+)h(\d+)', texto)
    doc = nlp(texto) 

    # Mapear granularidades de tempo para minutos
    #  mapeamento_granularidades = {'horas': 60, 'hora': 60, 'h': 60, 'minutos': 1, 'minuto': 1, 'min' : 1, 'dias': 480, 'dia': 480, 'd':480, 'semanas':2400, 'semana':2400, 's':2400, 'mes': 9600, 'meses': 9600}
    mapeamento_granularidades = {'horas': 60, 'hora': 60, 'h': 60, 'dias': 480, 'dia': 480, 'd':480}

    granularidade_encontrada = None
    valor_encontrado = 0

    #Extrair valores numéricos e inputando em lista
    numeros = [int(m) for m in re.findall(r'\d+', str(texto))]

    if numeros: #Tratamento para texto com valores numéricos           
            
        if len(numeros)>1: # Tratamento para textos com mais de 1 valor numerico, calculando a media dos valores antes do tratamento

            # Lista para armazenar os valores capturados de textos com mais de 1 valor numerico
            valores_encontrados = []
            cont = 0
            granularidade_encontrada = ''

            if match_hora_minuto:
                horas = int(match.group(1))
                minutos = int(match.group(3))
                valor_encontrado = (horas * 60) + minutos
                granularidade_encontrada = mapeamento_granularidades.get('minutos', 1)                
                valores_encontrados.append(valor_encontrado)
            
            elif match:              
                for i, grupo in enumerate(match.groups(), 1):
                    if texto.isdigit():
                        valores_encontrados.append(int(match.group(i)))
                    elif cont == 0:
                        cont =+ 1
                        granularidade_encontrada = match.group(i)
            
            if granularidade_encontrada and not len(valores_encontrados):
                print('padrao nao tratado: ', texto)
                return 0
            else:
                return sum(valores_encontrados) / len(valores_encontrados) * mapeamento_granularidades.get(granularidade_encontrada, 1) if granularidade_encontrada else (sum(numeros) / len(numeros))
        
        else:
            if match: #Identificacao de granularidade e valores do motor 1
                valor_encontrado = int(match.group(1))
                granularidade_encontrada = match.group(2)
                
                if granularidade_encontrada:  #retorna resposta do motor 1
                    # Calcular total em minutos e retornar valor caso granularidade tenha sido encontrada
                    return valor_encontrado * mapeamento_granularidades.get(granularidade_encontrada, 1)
            
            else: # Motor 2 de Identificacao de granularidade e valores 
                for token in doc:
                    if token.lemma_ in mapeamento_granularidades:
                        granularidade_encontrada = token.lemma_
                    elif token.is_digit:
                        valor_encontrado = int(token.text)
                
                # Calcular total em minutos
                return valor_encontrado * mapeamento_granularidades.get(granularidade_encontrada, 1) if granularidade_encontrada else valor_encontrado
    
    else: # Tratamento para texto sem valores numéricos, retorno = 0
        return 0
        
    texto=''
    match = ''
    granularidade_encontrada = None
    valor_encontrado = 0
    valor_encontrado_media = 0

# INPUTANDO DADOS

In [7]:
# necessario alterar path de acordo com o repositorio de execucao, lembrando de inverter contrabarra(\) para barra(/)
input_path = 'C:/Users/frederick.barros/Documents/PROJETOS/UNILEVER/unilever_dados/bucket/inputs/'
input_file_name_base_consolidada = 'Atena 2.0 _ Tempos & Movimentos _ Prévia de Resultado.xlsx'
input_sheet_name_base_consolidada = 'Base Consolidada'

#1 LENDO BASE DE DADOS A PARTIR DA PLANILHA Atena 2.0 _ Tempos & Movimentos _ Prévia de Resultado.xlsx para dataframe pandas
df = pandas.read_excel(input_path + input_file_name_base_consolidada, sheet_name=input_sheet_name_base_consolidada, engine='openpyxl')

##### CLASSIFICACAO DE EXECUTIVOS

In [8]:
input_file_name_classificacao_executivos = 'Executivos vs Tipo de Cliente.xlsx'
input_sheet_name_classificacao_executivos = 'Plano de Campo'

#1 LENDO BASE DE DADOS DE_PARA CLASSIFICACAO DE EXECUTIVOS
df_class_exe = pandas.read_excel(input_path + input_file_name_classificacao_executivos, sheet_name=input_sheet_name_classificacao_executivos, engine='openpyxl')

##### IDENTIFICACAO DE BU

In [9]:
input_file_name_bu_executivos = 'Plano de Campo _ Dez 2023.xlsx'
input_sheet_name_bu_executivos = 'Plano de Campo _ Dez_2023'

#1 LENDO BASE DE DADOS DE_PARA IDENTIFICACAO DE BU
df_bu_exe = pandas.read_excel(input_path + input_file_name_bu_executivos, sheet_name=input_sheet_name_bu_executivos, engine='openpyxl')

##### MAPPING DE ATIVIDADES

In [10]:
input_file_name_atividades = 'Relação - Etapa X Macro Atividade X Atividades.xlsx'
input_sheet_name_atividades = 'sheet1'
#input_sheet_name_atividades = 'mapping atividades'

df_atividades = pandas.read_excel(input_path + input_file_name_atividades, sheet_name=input_sheet_name_atividades, engine='openpyxl')

df_atividades = df_atividades.dropna(subset=['ATIVIDADES'])

In [11]:
def preencher_valores_nulos(df):
    # Preencher valores nulos nas colunas "etapa" e "macro_atividade" com os valores da linha anterior
    df['ETAPA'].fillna(method='ffill', inplace=True)
    df['MACRO ATIVIDADE'].fillna(method='ffill', inplace=True)

preencher_valores_nulos(df_atividades)

novos_nomes = ['etapa','macro_atividade','atividade']
renomear_colunas_por_lista(df_atividades, novos_nomes)

df_atividades = df_atividades.map(limpa_texto)

df_atividades['atividade_ajustada']=df_atividades['atividade'].apply(tratar_valor)

df_atividades.insert(loc=0, column='id', value=range(0, 0 + len(df_atividades)))

pandas.set_option('display.max_colwidth', None)

C:\Users\frederick.barros\AppData\Local\Temp\ipykernel_17852\3450515245.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['ETAPA'].fillna(method='ffill', inplace=True)
C:\Users\frederick.barros\AppData\Local\Temp\ipykernel_17852\3450515245.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['MACRO ATIVIDADE'].fillna(method='ffill', inplace=True)


In [12]:
#display(df_ind_vendas)

# TRATAMENTOS

### TRATATIVA 1: PREPARANDO DATA SETS PARA TRATAMENTO

In [13]:
# ADICIONANDO ID AO DATASET PRINCIPAL PARA OBTERMOS CHAVE DE CRUZAMENTO
df.insert(loc=0, column='ID', value=range(0, 0 + len(df)))

In [14]:
# Carregar o modelo do português do Brasil
nlp = spacy.load("pt_core_news_sm")

##### CABECALHO

In [15]:
df_cabecalho = df[["ID", "PARTICIPANTE", "CARGO", "REGIONAL", "CANAL DE ATENDIMENTO", "CANAL AJUSTADO"]]

In [16]:
# aplicando a funcao de limpeza às colunas do tipo texto
colunas_texto = df_cabecalho.select_dtypes(include=['object']).columns
df_cabecalho[colunas_texto] = df_cabecalho[colunas_texto].applymap(limpa_texto)

C:\Users\frederick.barros\AppData\Local\Temp\ipykernel_17852\105214288.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cabecalho[colunas_texto] = df_cabecalho[colunas_texto].applymap(limpa_texto)
C:\Users\frederick.barros\AppData\Local\Temp\ipykernel_17852\105214288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cabecalho[colunas_texto] = df_cabecalho[colunas_texto].applymap(limpa_texto)


##### CLASSIFICACAO DE EXECUTIVOS

In [17]:
df_class_exe = df_class_exe[["EXECUTIVO DE NEGÓCIO", "TIPO DE CLIENTE"]].drop_duplicates()

In [18]:
df_class_exe = df_class_exe.map(limpa_texto)

In [19]:
df_class_exe = df_class_exe.groupby('EXECUTIVO DE NEGÓCIO')['TIPO DE CLIENTE'].agg(lambda x: ' | '.join(x)).reset_index()

##### BU DE EXECUTIVOS

In [20]:
df_bu_exe = df_bu_exe[["NOME EXECUTIVO DE NEGÓCIO", "BU"]].drop_duplicates()

In [21]:
df_bu_exe = df_bu_exe.map(limpa_texto)

In [22]:
df_bu_exe = df_bu_exe.groupby('NOME EXECUTIVO DE NEGÓCIO')['BU'].agg(lambda x: ' | '.join(x)).reset_index()

### TRATATIVA 2: Atividades anuais
##### converter valor informado (em minutos) para horas - dividir por 60
Resultado esperado: horas/ano##### 
Esta regra se aplica às colunas: G, H, I,J


In [23]:
colunas_selecionadas = ['ID'
                        ,'(JBP) Quanto tempo (em minutos) você gasta no desdobramento e/ou customização da estratégia do JBP anual para o(s) seu(s) cliente(s) ? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'(JBP) Quanto tempo (em minutos) você gasta com a geração de relatórios e com a preparação da apresentação do JBP anual para o(s) seu(s) cliente(s)? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'(JBP) Quanto tempo (em minutos) você gasta com a apresentação do JBP anual para o(s) seu(s) cliente(s)? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'(JBP) Após finalização do processo de JBP anual, quanto tempo (em minutos) você gasta na consolidação, formalização e registro das metas acordadas com o(s) cliente(s)? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                       ]
df_t2 = df.loc[:, colunas_selecionadas].copy()

# display(df_t2)

In [24]:
# novos_nomes = ['id','desdobramento_e_ou_customizacao_da_estrategia_do_jbp', 'tempo_dedicado_as_lives_de_promocoes_', 'apresentacao_do_jbp', 'consolidacao_formalizacao_e_registro_das_metas_acordadas_com_os_clientes']

In [25]:
# renomeado colunas com o nome das colunas no arquivo de mapping
novos_nomes = [
    'id',
    'Desdobramento e/ou customização da estratégia do JBP',
    'Geração de relatórios e preparação da apresentação do JBP',
    'Apresentação do JBP',
    'Consolidação, formalização e registro das metas acordadas com o(s) cliente(s)'
]
renomear_colunas_por_lista(df_t2, novos_nomes)

# padronizando nomes colunas
df_t2 = renomear_colunas_tratando_nome(df_t2)

# renomear_colunas_por_lista(df_t2, novos_nomes)
# display(df_t2)

In [26]:
df_t2 = df_t2.map(calcula_minutos)
#display(df_t2)

### TRATATIVA 3: Atividades anuais realizadas mais de uma vez ao ano
Multiplicar o tempo informado em minutos pela quantidade de vezes que a atividade é realizada. O resultado será o tempo em min/ano.
Este valor deverá ser convertido em horas
##### Resultado esperado: horas/ano
Esta regra se aplica às colunas: K/L (coluna K = tempo gasto / Coluna L = quantidade de vezes em que a atividade é realizada), AQ/AR (coluna AQ = tempo gasto / Coluna AR = quantidade de vezes em que a atividade é realizada), FK/FL (coluna FK = tempo gasto / coluna FL = quantidade de vezes em que a atividade é realizada)


In [27]:
colunas_selecionadas = ['ID'
                        ,'(JBP) Quanto tempo (em minutos) você gasta em cada ciclo de revisão do JBP? Preparação material, apresentação e revisão de metas etc. Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'(JBP)Quantas vezes você faz ao ano as revisões do JBP?'
                        ,'Quanto tempo, por mês, (em minutos) você gasta preparando e enviando tabelas de preço para os clientes.?'
                        ,'Quantos meses, ao longo do ano, você faz atualizações de tabelas de preço para os clientes?'
                        ,'Quanto tempo, (em minutos) você dedica para cada exame ocupacional da Unilever?'
                        ,'Quantas vezes, ao longo do ano, você faz o exame ocupacional da Unilever?'
                       ]
df_t3 = df.loc[:, colunas_selecionadas].copy()
#df_t3 = renomear_colunas(df_t3)

novos_nomes = ['id',
               'v1',
               'qtd1',
               'v2',
               'qtd2',
               'v3',
               'qtd3']
renomear_colunas_por_lista(df_t3,novos_nomes)

# display(df_t3)

,id,v1,qtd1,v2,qtd2,v3,qtd3
0,0,480,3 meses,60,4,90,1
1,1,1440,2 meses,300,12,360,1
2,2,400,4 meses,80,12,600,1
3,3,60,2 meses,30,12,179,1
4,4,2870,3 meses,330,12,0,0
...,...,...,...,...,...,...,...
378,378,480,3,30,12,120,1
379,379,480,4,120,6,120,1
380,380,480,3,1800,12,90,1
381,381,480 minutos,2,300,4,60,1


In [28]:
df_t3 = df_t3.map(calcula_minutos)
#display(df_t3)

In [29]:
# converter horas/ano - tempo gasto / quantidade de vezes em que a atividade é realizada - dividir por 60
# Criando novas colunas resultantes das divisões
df_t3['Ciclo de revisão do JBP (preparação material, apresentação e revisão de metas etc.)'] = np.where(
    df_t3['qtd1'] > 0,
    df_t3['v1'] *
    df_t3['qtd1'],
    df_t3['v1']
)

df_t3['Preparo e envio de tabelas de preços aos clientes'] = np.where(
    df_t3['qtd2'] > 0,
    df_t3['v2'] *
    df_t3['qtd2'],
    df_t3['v2']
)

df_t3['Exame ocupacional da Unilever'] = np.where(
    df_t3['qtd3'] > 0,
    df_t3['v3'] *
    df_t3['qtd3'],
    df_t3['v3']
)

In [30]:
df_t3 = renomear_colunas_tratando_nome(df_t3)

# display(df_t3)

In [31]:
# Selecionar as colunas desejadas para o t3
colunas_selecionadas = ['id', 
                        'ciclo_de_revisao_do_jbp_preparacao_material_apresentacao_e_revisao_de_metas_etc', 
                        'preparo_e_envio_de_tabelas_de_precos_aos_clientes', 
                        'exame_ocupacional_da_unilever']

# Criar o novo DataFrame com as colunas selecionadas
df_t3 = df_t3.loc[:, colunas_selecionadas].copy()


# display(df_t3)

### TRATATIVA 4: Atividades com tempo estimado mensal
São perguntas que já solicitaram ao usuário o tempo médio gasto ao longo do mês. Para estas perguntas é necessário apenas converter o valor informado para horas
##### Resultado esperado: horas/mês
Esta regra se aplica às colunas: BO, BP, BQ, BT, ER, FG, FH, EI


In [32]:
nome_coluna_er = df.columns[149]
nome_coluna_ei = df.columns[139]
#print(nome_coluna_ei)
colunas_selecionadas = ['ID'
                        ,'Quanto tempo, por mês (em minutos), você gasta realizando a gestão do portfólio com o cliente?'
                        ,'Quanto tempo, por mês (em minutos), você gasta ajustando o trade story de cada inovação para levar o material compilado para apresentar para o comprador? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'Quanto tempo, por mês (em minutos), você gasta se preparando para comprovação dos planos de trade? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        ,'Quanto tempo, por mês (em minutos), você gasta para preencher e validar as ações no HUB de finanças? Caso tenha mais de um cliente, considere o tempo total gasto com todos os clientes'
                        , nome_coluna_er
                        ,'Quanto tempo, por mês (em minutos) você dedica em lives de promoções?'
                        ,'Quanto tempo, por mês, (em minutos) você dedica em ações do Dia D?'
                        ,nome_coluna_ei
                       ]
df_t4 = df.loc[:, colunas_selecionadas].copy()
#df_t4 = renomear_colunas(df_t4)
novos_nomes = ['id',
               'gestao_de_portfolio_com_o_cliente', 
               'ajuste_de_trade_story_de_inovacoes_para_apresentacao_do_material_ao_comprador', 
               'preparacao_para_comprovacao_dos_planos_de_trade', 
               'preenchimento_e_validacao_das_acoes_no_hub_de_financas', 
               'treinamentos_fornecidos_pela_unilever', 
               'tempo_dedicado_as_lives_de_promocoes', 
               'tempo_dedicado_as_acoes_do_dia_d', 
               'atividades_extras_projetos_internos_da_unilever_liderados_por_regional']
renomear_colunas_por_lista(df_t4, novos_nomes)
# display(df_t4)

,id,gestao_de_portfolio_com_o_cliente,ajuste_de_trade_story_de_inovacoes_para_apresentacao_do_material_ao_comprador,preparacao_para_comprovacao_dos_planos_de_trade,preenchimento_e_validacao_das_acoes_no_hub_de_financas,treinamentos_fornecidos_pela_unilever,tempo_dedicado_as_lives_de_promocoes,tempo_dedicado_as_acoes_do_dia_d,atividades_extras_projetos_internos_da_unilever_liderados_por_regional
0,0,90,61,93,61,90,90,60,0
1,1,300,360,360,90,90,120,960,960
2,2,65,54,600,301,44,302,840,50
3,3,42,20,30,22,120,61,60,30
4,4,361,595,323,66,308,63,410,120
...,...,...,...,...,...,...,...,...,...
378,378,30,120,120,20,120,45,60,120
379,379,120,180,180,0,60,30,240,30
380,380,520,400,300,0,130,90,900,0
381,381,120 minutos,180 minutos,240 minutos,300 minutos,90,240,480,0


In [33]:
df_t4 = df_t4.map(calcula_minutos)
# display(df_t4)

padrao nao tratado:  300mimutos300


In [34]:
# # excluindo a coluna ID para aplicar a funcao no df_t2
# df_t4.iloc[:, 1:] = df_t4.iloc[:, 1:].map(converte_minutos_em_horas)
# display(df_t4)

### TRATATIVA 5: CONVERSAO MINUTOS POR MES
 Para todas as demais atividades o cálculo a ser feito é o de horas/mês gasto na atividade. 
 Existem dados recebidos via form que contem valores de "minutos em intervalo de semanas"
 e precisamos desses dados na granularidade de "minutos/mes"
##### REGRA DE CALCULO:
 1. Se o campo "minutos/semana" possuir valor > 0 e o campo "semanas" nao possuir valor ou possuir valor 0, 
 consideraremos como uma semana de trabalho e faremos o calculo da metrica mensal multiplicando o campo "minutos/semana" por 4 
2. Se o campo "minutos/semana" possuir valor > 0 e o campo "semanas" nao possuir valor ou possuir valor 0, consideraremos como uma semana

In [35]:
# adicionando a pocicao de todas as colunas exceto as de 
# Atividades anuais (G, H, I, J)
# Atividades anuais realizadas mais de uma vez ao ano (K, L, AQ, AR, FK, FL)
# Atividades com tempo estimado mensal (BO, BP, BQ, BT, ER, FG, FH)
# print(df.shape)
# lista de ranges que nao compreendem os dados acima
indices_colunas_selecionadas = [0] + list(range(13, 43)) + list(range(45,67)) + list(range(70,72)) + list(range(73,139)) + list(range(140,148)) + list(range(149,163)) + list(range(165,167)) + list(range(169,173))
#print(indices_colunas_selecionadas)
df_t5 = df.iloc[:, indices_colunas_selecionadas].copy()
# df_t5 = renomear_colunas(df_t5)
# print(df_t5.shape[1])

In [36]:
%%time
df_t5 = df_t5.map(calcula_minutos)
# display(df_t5)

padrao nao tratado:  60i0minutos
CPU times: total: 5min 51s
Wall time: 5min 58s


In [37]:
# Mantendo a primeira coluna 'ID'
df_t5_result = df_t5[['ID']].copy()
qtd_colunas= int(df_t5.shape[1])-1
for i in range(1, qtd_colunas, 2):  # Começa do segundo campo (índice 1) e vai até o décimo campo (índice 19)
    #df_t5_result[f'col{i} x col{i+1}'] = df_t5.iloc[:, i] * df_t5.iloc[:, i+1] if df_t5.iloc[:, i+1] > 0 else df_t5.iloc[:, i]
    if i:
        col_atual = df_t5.columns[i]
        col_seguinte = df_t5.columns[i+1]
        nova_coluna = f'{col_atual}_X_{col_seguinte}' if col_seguinte else col_atual
        
        df_t5_result[nova_coluna] = np.where(
            df_t5.iloc[:, i+1] > 0,
            (df_t5.iloc[:, i] * df_t5.iloc[:, i+1]),
            df_t5.iloc[:, i]
        )

# print(int(df_t5_result.shape[1]))
# display(df_t5_result)

In [38]:
novos_nomes = ['id',
               'acompanhamento_execucao_das_acoes_de_jbp', 
               'planejando_acoes_do_compre_agora', 
               'acompanhamento_de_acoes_do_compre_agora', 
               'planejamento_e_analise_dos_programas_efetividade_superacao_e_levers2', 
               'acompanhamento_do_programa_fidelidade_do_cliente_tradecash', 
               'planejamento_e_analise_do_fac', 'analise_de_tendencias_de_sell_out', 
               'planejamento_de_acoes_de_sell_out_como_tabloides_festivais_rebaixas_etc', 
               'analise_de_rupturas_e_estoque', 
               'atualizacao_de_indicadoresdados_de_vendas_do_mes_anterior_sell_in_sell_out_share_exececucao_etc', 
               'analise_de_comportamento_de_share', 
               'desenhoaprovacao_de_planos_de_acao_atrelado_ao_comportamento_de_share', 
               'analise_de_indicadores_de_programas_vpv_e_vpl', 
               'analise_de_outros_indicadores_de_vendas', 
               'download_da_lista_de_precos_e_analises_adicionais', 
               'analisando_sua_estimativa_e_elaborando_sua_proposta_para_pedidos_do_mes', 
               'acompanhamento_de_metas_definidas_no_jbp', 
               'preenchimento_da_ferramenta_my_planner_metas_e_prioridades', 
               'consulta_de_dados_no_pgv', 'procura_e_acesso_a_dados_fora_do_pgv', 'solicitacao_de_dados_e_relatorios_para_outras_areas', 'validacao_de_prioridades_mensais_com_a_lideranca', 'registro_de_resultados_do_planejamento_mensal', 'solicitacao_de_imagens_e_artes_para_acoes_de_trade_tabloides_ecommerce_cadastro_de_produtos_etc', 'controle_de_investimentos_dos_clientes', 'acompanhamento_de_cadastro_positivacao_pipe_simplus_dados_fiscais_das_inovacoes', 'criacao_e_envio_de_proposta_de_acoes_para_trade_regional', 'concilar_conta_corrente_cliente_ppm_forms_apuracoes_encontro_de_contas_e_etc', 'tempo_de_espera_para_ser_atendido_pelo_cliente', 'tempo_de_atendimento_pelo_cliente', 'tempo_em_reunioes_com_o_cliente', 'preparacao_e_construcao_da_narrativa_de_venda_ao_cliente', 'analise_de_pedidos_edi_e_possiveis_ajustes_de_precopedido_p_cada_comprador', 'tempo_gasto_em_reunioes_com_outras_areas_do_cliente', 'tempo_gasto_nutrindo_o_relacionamento_com_o_cliente_fora_negociacao', 'busca_por_ajuda_e_resolucao_de_problemas_fiscaisprecoscondicoes_de_pagamento_erradas', 'resposta_a_cotacao_de_preco_dos_clientes_farma_regional', 'insercao_de_pedidos_no_sistema_salesforce', 'validacao_confirmacao_do_pedido_no_salesforce', 'tratarmontar_o_sua_base_excel_antes_de_subir_seu_pedido_no_pharmalink_clientes_nao_edi', 'insercao_de_pedidos_no_sistema_pharmalink_clientes_nao_edi', 'monitoramento_da_base_de_pedidos_para_os_clientes', 'busca_por_informacoes_sobre_atendimento_pstk_previsibilidade_de_estoque', 'controle_de_entrada_faturamento_das_ofertas_acompanhamento_cota', 'conferencia_do_boletim_diario_de_entrada_faturado_estimativa', 'prorrogacao_de_notas_negociadas_com_o_cliente', 'analise_de_carteira_div_preco_status_pedido_formacoes_agendas_fechamento_de_carga_etc', 'demandas_de_notas_de_devolucao_e_devolucao_parcial', 'tempo_dedicado_com_interface_gs_e_negociacao_de_janelas_de_entregas_ou_resolvendo_problemas_de_entregas', 'fechamento_e_acompanhamento_de_overflow', 'analise_de_relatorio_semanal_de_notas_vencidas_e_busca_de_resolucao_com_cliente_gs_e_area_cobranca_unilever', 'acompanhamento_de_precosacoes_da_concorrencia', 'deslocamento_a_trabalho_visitas_a_clientes_reunioes_em_geral_rota_a_loja_etc', 'tempo_de_trade_visit_em_loja', 'registro_geracao_de_reports_das_visitas_a_loja', 'acompanhamento_dos_pontos_reportados_na_visita_a_loja', 'consulta_de_informacoes_previas_sobre_a_loja_a_ser_visitada', 'conversas_com_o_time_de_merchandising_nas_visitas', 'gerar_acoes_para_os_pontos_de_melhoria_e_gaps_encontrados_nas_lojas', 'relacionamento_com_o_time_do_cliente_loja_encarregados_e_gerentes', 'tempo_dedicado_para_demandas_da_area_de_trade_regional_preenchimento_relatorios_estoques_sell_out_e_outros_acompanhamentos_solicitados_pelas_categorias', 'participacao_em_reunioes_de_outras_areas', 'tempo_dedicado_para_desenvolvimento_pessoal_e_profissional', 'lancamento_de_notas_fiscais_referentes_aos_gastos_do_mes_rdv', 'reunioes_entre_lideranca_e_liderados_para_planejamento_acompanhamento_e_apresentacao_de_resultados', 'tempo_dedicado_a_pesquisas_organizacionais', 'treinamento_coaching_com_time_de_distribuidores', 'reunioes_de_vendas_internas_na_unilever_regionais_e_ou_nacionais', 'leitura_e_resposta_de_emails', 'leitura_e_respostas_de_mensagens_no_whatsapp', 'reunioes_com_os_times_de_trade_region_categoria_e_canal', 'analise_e_repasse_da_apuracao_do_pack_comercial_ao_cliente', 'demandas_diversas_do_cliente', 'demandas_extras_da_regional']

renomear_colunas_por_lista(df_t5_result, novos_nomes)

# display(df_t5_result)

,id,acompanhamento_execucao_das_acoes_de_jbp,planejando_acoes_do_compre_agora,acompanhamento_de_acoes_do_compre_agora,planejamento_e_analise_dos_programas_efetividade_superacao_e_levers2,acompanhamento_do_programa_fidelidade_do_cliente_tradecash,planejamento_e_analise_do_fac,analise_de_tendencias_de_sell_out,planejamento_de_acoes_de_sell_out_como_tabloides_festivais_rebaixas_etc,analise_de_rupturas_e_estoque,...,reunioes_entre_lideranca_e_liderados_para_planejamento_acompanhamento_e_apresentacao_de_resultados,tempo_dedicado_a_pesquisas_organizacionais,treinamento_coaching_com_time_de_distribuidores,reunioes_de_vendas_internas_na_unilever_regionais_e_ou_nacionais,leitura_e_resposta_de_emails,leitura_e_respostas_de_mensagens_no_whatsapp,reunioes_com_os_times_de_trade_region_categoria_e_canal,analise_e_repasse_da_apuracao_do_pack_comercial_ao_cliente,demandas_diversas_do_cliente,demandas_extras_da_regional
0,0,180.0,0,0,0.0,0.0,0.0,480.0,720.0,960.0,...,90.0,60,0,180.0,240.0,364,180,60,360,180
1,1,480.0,0,0,0.0,0.0,0.0,1920.0,1920.0,1920.0,...,270.0,90,0,0.0,1800.0,1800,0,90,360,120
2,2,120.0,80,100,100.0,100.0,81.0,968.0,750.0,780.0,...,44.0,61,0,320.0,496.0,2400,77,122,3200,1220
3,3,45.0,10,22,11.0,11.0,10.0,121.0,120.0,82.0,...,240.0,63,1,10.0,120.0,252,59,46,40,486
4,4,485.0,600,281,315.0,321.0,589.0,1124.0,1022.0,556.0,...,308.0,0,0,60.0,684.0,384,61,121,1360,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,378,60.0,0,0,0.0,0.0,960.0,1440.0,720.0,640.0,...,240.0,30,480,840.0,1200.0,4800,540,120,640,360
379,379,60.0,0,0,0.0,0.0,0.0,360.0,960.0,540.0,...,240.0,60,0,720.0,2400.0,2400,0,120,1200,1200
380,380,480.0,0,0,0.0,720.0,30.0,4000.0,1800.0,1920.0,...,390.0,60,600,120.0,4800.0,8800,60,400,3600,60
381,381,120.0,0,0,180.0,120.0,120.0,480.0,1920.0,1920.0,...,180.0,0,0,960.0,1200.0,4800,0,60,1200,0


# MONTANDO CABECALHO

##### ADICIONANDO DADOS DE CLASSIFICACAO EXECUTIVO AO CABECALHO

In [39]:
#regra: adcionar N/A aos valores nao encontrados de tipo cliente a partir do df_cabecalho (df_cabecalho left join df_class_exe)
df_cabecalho = pandas.merge(df_cabecalho, df_class_exe, left_on='PARTICIPANTE', right_on='EXECUTIVO DE NEGÓCIO', how='left').drop('EXECUTIVO DE NEGÓCIO', axis=1)

##### ADICIONANDO DADOS DE CLASSIFICACAO E BU DOS EXECUTIVO AO CABECALHO

In [40]:
#regra: adcionar N/A aos valores nao encontrados de tipo cliente a partir do df_cabecalho (df_cabecalho left join df_class_exe)
df_cabecalho = pandas.merge(df_cabecalho, df_bu_exe, left_on='PARTICIPANTE', right_on='NOME EXECUTIVO DE NEGÓCIO', how='left').drop('NOME EXECUTIVO DE NEGÓCIO', axis=1)

In [41]:
novos_nomes = ['id','participante', 'cargo', 'regional', 'canal_de_atendimento', 'canal_ajustado', 'tipo_de_cliente', 'bu']
renomear_colunas_por_lista(df_cabecalho, novos_nomes)

# display(df_cabecalho)

,id,participante,cargo,regional,canal_de_atendimento,canal_ajustado,tipo_de_cliente,bu
0,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw
1,1,marcos paulo alves moura,executivo de vendas,sao paulo,varejo direto,varejo direto,bronze | gold,pc | bw
2,2,luiz rodrigo dumer santos,executivo de vendas,leste,varejo direto | ecommerce | varejo indireto,misto,bronze | gold | silver,fr | hc | mt
3,3,jefferson pinheiro queiroz,executivo de vendas,nordeste,varejo indireto,varejo indireto,gold,pc | bw
4,4,julio cesar da silva,executivo de vendas,leste,farma indireto,farma indireto,silver,pc | bw
...,...,...,...,...,...,...,...,...
378,378,anonymous,lider de vendas,rota da beleza,farma indireto,farma indireto,NaN,NaN
379,379,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN
380,380,anonymous,executivo de vendas,sul,farma indireto,farma indireto,NaN,NaN
381,381,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN


# MONTANDO DADOS CALCULADOS

##### ADICIONANDO DADOS TRATAMENTO 2 e 3

In [42]:
df_dados = pandas.merge(df_t2, df_t3, on="id")

##### ADICIONANDO DADOS TRATAMENTO 4

In [43]:
df_dados = pandas.merge(df_dados, df_t4, on="id")

##### ADICIONANDO DADOS TRATAMENTO 5

In [44]:
# df_dados = pandas.merge(df_dados, df_t5_result, on="calc_id")
# unindo dataframe de cabeclaho e de dados
df_dados = pandas.merge(df_dados, df_t5_result, on="id")

In [45]:
#display(df_dados)

# OUTPUT

##### MONTANDO DATASET FINAL

In [46]:
# unindo dataframe de cabeclaho e de dados
df_base_consolidada_tratada_MIN = pandas.merge(df_cabecalho, df_dados, on='id', how='left')
# df_base_consolidada_tratada_MIN = df_base_consolidada_tratada_MIN.rename(columns={'ID_X': 'ID'}, inplace=True)
# df_base_consolidada_tratada_MIN = df_base_consolidada_tratada_MIN.drop(['calc_id', 'ID_y'], axis=1, inplace=True)

# CRIANDO BASE CONVERTIDA EM HORAS

In [47]:
df_dados_horas = df_dados.copy()

In [48]:
def converte_minutos_em_horas(df):
    # Verifica se o DataFrame é não vazio
    if not df.empty:
        # Itera sobre as colunas do DataFrame
        for coluna in df.columns:
            # Verifica se o nome da coluna não é 'ID'
            if (coluna != 'id')  & (coluna != 'ID')  & (coluna != 'ID_x') &  (coluna != 'ID_y') & (coluna != 'calc_ID'):
                # Verifica se os valores são maiores que zero antes de dividir por 60
                df[coluna] = df[coluna].apply(lambda x: x / 60 if x > 0 else x)
            else:
                print(coluna)
    else:
        print("O DataFrame está vazio.")
    return df

df_dados_horas = converte_minutos_em_horas(df_dados_horas)

id


In [49]:
# unindo dataframe de cabeclaho e de dados
df_base_consolidada_tratada_HR = pandas.merge(df_cabecalho, df_dados_horas, on='id', how='left')

#display(df_base_consolidada_tratada_HR)

# CALCULANDO OUTLIERS
transformar e cruzar dados com input de percepcoes de outlier
o executivo que possuir mais de um canal de vendas sera considerado misto

In [50]:
# Transpondo df_dados de forma a gerar novo df com 3 colunas e transposicao de colunas para linhas, mantendo campo ID
df_dados_transpostos = pandas.melt(df_dados, id_vars=['id'], var_name='atividade', value_name='valor').drop_duplicates()

# Juntando cabecalho aos dados transpostos
df_transpostos = pandas.merge(df_cabecalho, df_dados_transpostos, on='id', how='left')

#renomeando campo id para id participante
df_transpostos = df_transpostos.rename(columns={'id': 'id_participante', 'atividade': 'atividade_ajustada'}).drop_duplicates()

# display(df_outliers_cabecalho)
# display(df_transpostos.sort_values(by='id_participante'))

In [51]:
#df_outliers = df_outliers.drop(['id_atividade', 'percep_atividade', 'canal','atividade_ajustada', 'etapa', 'macro_atividade'], axis=1)


##### ADICIONANDO ETAPA E MACRO ETAPA DA ATIVIDADE

In [52]:
df_atividade_copy = df_atividades.copy()
df_atividade_copy = df_atividade_copy.rename(columns={'id': 'id_atividade'}).drop_duplicates()

# resetando indices para preparar join
df_transpostos.reset_index(drop=True, inplace=True)
df_atividade_copy.reset_index(drop=True, inplace=True)

df_final = pandas.merge(df_transpostos, df_atividade_copy, on=['atividade_ajustada'], how='left').drop_duplicates()#.drop('atividade', axis=1)

# removendo campos ao utilizados
# df_outliers = df_outliers.drop(['percep_atividade', 'canal','atividade_ajustada'], axis=1)

# display(df_final)

In [53]:
#

In [54]:
# display(df_outliers_final)
df_print = df_final[df_final['id_atividade'].isnull()]
# df_print = df_final[['id_atividade', 'atividade']].drop_duplicates()
df_print = df_print.reset_index()

if len(df_print) > 0:
    print('Erro: atividade sem identificacao')
    display(df_print)
    # input("Pressione Enter para continuar...")

# ULTIMOS DETALHES

In [55]:
# Definir a ordem desejada das colunas
nova_ordem_colunas = ['id_participante', 'participante', 'cargo', 'regional' ,'canal_de_atendimento' ,'canal_ajustado' ,'tipo_de_cliente' ,'bu' ,'etapa' ,'macro_atividade' , 'id_atividade', 'atividade' ,'atividade_ajustada' ,'valor']
df_final = df_final[nova_ordem_colunas]

# Criando campo calculado em horas
df_final['valor_horas'] = df_final['valor'] / 60

display(df_final)

,id_participante,participante,cargo,regional,canal_de_atendimento,canal_ajustado,tipo_de_cliente,bu,etapa,macro_atividade,id_atividade,atividade,atividade_ajustada,valor,valor_horas
0,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw,planejamento anual,jbp,0,desdobramento eou customizacao da estrategia do jbp,desdobramento_e_ou_customizacao_da_estrategia_do_jbp,480.0,8.0
1,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw,planejamento anual,jbp,1,geracao de relatorios e preparacao da apresentacao do jbp,geracao_de_relatorios_e_preparacao_da_apresentacao_do_jbp,480.0,8.0
2,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw,planejamento anual,jbp,2,apresentacao do jbp,apresentacao_do_jbp,480.0,8.0
3,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw,planejamento anual,jbp,3,consolidacao formalizacao e registro das metas acordadas com os clientes,consolidacao_formalizacao_e_registro_das_metas_acordadas_com_os_clientes,480.0,8.0
4,0,valdir domingos dos santos,executivo de vendas,sao paulo,varejo direto,varejo direto,gold,pc | bw,planejamento anual,jbp,4,ciclo de revisao do jbp preparacao material apresentacao e revisao de metas etc,ciclo_de_revisao_do_jbp_preparacao_material_apresentacao_e_revisao_de_metas_etc,1440.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34082,382,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN,outras atividades,fac,81,leitura e respostas de mensagens no whatsapp,leitura_e_respostas_de_mensagens_no_whatsapp,360.0,6.0
34083,382,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN,outras atividades,fac,82,reunioes com os times de trade region categoria e canal,reunioes_com_os_times_de_trade_region_categoria_e_canal,180.0,3.0
34084,382,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN,outras atividades,fac,85,analise e repasse da apuracao do pack comercial ao cliente,analise_e_repasse_da_apuracao_do_pack_comercial_ao_cliente,90.0,1.5
34085,382,anonymous,executivo de vendas,sao paulo,varejo direto,farma indireto,NaN,NaN,outras atividades,fac,87,demandas diversas do cliente,demandas_diversas_do_cliente,120.0,2.0


# ESCREVENDO DADO EM XLSX

In [56]:
# ESCREVENDO NOVA SHEET EM UM ARQUIVO DE OUTPUT (NOVO ARQUIVO)
output_path = 'C:/Users/frederick.barros/Documents/PROJETOS/UNILEVER/unilever_dados/bucket/outputs/'
output_file_name = 'v8_unilever_dados_atena_tempos_e_movimentos.xlsx'
output_sheet_name_min = 'base_tratada_minutos'
output_sheet_name_hr = 'base_tratada_horas'
output_sheet_name_atv = 'mapping_atividades'
output_sheet_name_perc_out = 'percepcao_outliers_min'
output_sheet_name_desvio_padrao = 'desvio_padrao_min'
output_sheet_name_out = 'outliers_min'
output_sheet_name_final = 'base_pesquisa_dashboard'
#df_base_consolidada_tratada.to_excel(output_path + output_file_name, sheet_name=output_sheet_name)

In [57]:
# Escrevendo multiplas sheets
with pandas.ExcelWriter(output_path+output_file_name) as writer:
    df_base_consolidada_tratada_MIN.to_excel(writer, sheet_name=output_sheet_name_min)
    df_base_consolidada_tratada_HR.to_excel(writer, sheet_name=output_sheet_name_hr)
    df_atividades.to_excel(writer, sheet_name=output_sheet_name_atv)
    # df_percep_outliers.to_excel(writer, sheet_name=output_sheet_name_perc_out)
    # df_desvio_padrao.to_excel(writer, sheet_name=output_sheet_name_desvio_padrao)
    # df_outliers_final.to_excel(writer, sheet_name=output_sheet_name_out)
    df_final.to_excel(writer, sheet_name=output_sheet_name_final)